### Music recommender system

One of the most used machine learning algorithms is recommendation systems.  
  
A **recommender** (or recommendation) **system** (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.

#### Import required libraries

In [1]:
import numpy as np
import pandas as pd

#### Reading the files

In [21]:
#Read userid-songid-listen_count triplets
#This step might take time to download data from external sources
song_info_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

song_info = pd.read_csv(song_info_file,sep='\t',header=None)
song_info.columns = ['user_id', 'song_id', 'listen_count']

#Read song  metadata
song_actual =  pd.read_csv(songs_file)

#Merge the two dataframes above to create input dataframe for recommender systems
song_df = pd.merge(song_info, song_actual.drop_duplicates(['song_id']), on="song_id", how="left")